here we put together things from the nnet and neural architecture search notebooks, and see what happens

In [1]:
import pandas as pd
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import datetime
import time
from scipy.stats import probplot
import datetime
import seaborn as sns
sns.set()
from sklearn.utils import shuffle
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard, EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, BatchNormalization, GaussianNoise, Input, PReLU, Activation, Concatenate
from keras.initializers import VarianceScaling
from keras import regularizers 
from keras.models import load_model
from keras import backend as K
from sklearn import metrics
import joblib

/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv('data/cabauw/training-data.csv.gz')

In [3]:
class ControllerRNN:
    def __init__(self, max_len, batch_size, type_size, arg_size,
                 learning_rate=0.001, hidden_size=32, baseline_smoothing=0.95):
        self.hidden_size = hidden_size
        self.unroll_by = max_len
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.type_size = type_size + 1  # 0 is for end-of-network token
        self.arg_size = arg_size
        self.baseline_smoothing = baseline_smoothing

    def build(self):
        # reward for the architectures
        self.architecture_reward = tf.placeholder(tf.float32, [self.batch_size])
        
        # exponential moving average of the reward
        self.last_average_reward = tf.reduce_mean(self.architecture_reward)
        self.reward_ema = tf.train.ExponentialMovingAverage(self.baseline_smoothing)
        self.update_reward_ema = self.reward_ema.apply([self.last_average_reward])

        rnn = tf.contrib.rnn.GRUCell(self.hidden_size)
        state = tf.random_normal([self.batch_size, rnn.state_size])
        
        # weight matrices to transform from rnn output to layer type and discrete arg
        rnn_to_layer_type_weight = tf.Variable(tf.random_normal([rnn.output_size, self.type_size]))
        rnn_to_layer_type_gradient = []
        rnn_to_layer_arg_weight = tf.Variable(tf.random_normal([rnn.output_size, self.arg_size]))
        rnn_to_layer_arg_gradient = []

        # rnn output and gradients
        output = tf.random_normal([self.batch_size, rnn.output_size])

        # layer_probs contains the output from the network, namely the probabilities
        # of type and argument for every layer of every network
        self.layer_probs = []

        # layer_indicators contains one-hot indicators of type and argument
        # for every layer of every network.
        # used to select which action is used to compute the gradient.
        # fixed, must be set before updating the weights
        self.layer_indicators = []

        losses = []
        
        for i in range(self.unroll_by):
            # run rnn cell
            output, state = rnn(output, state)

            if i == 0:  # gru variables are only initialized now
                rnn_params = rnn.trainable_variables + rnn.trainable_weights
                rnn_gradients = [[] for _ in range(len(rnn_params))]

            # compute output probabilites
            layer_type = tf.nn.softmax(tf.matmul(output, rnn_to_layer_type_weight))
            layer_arg = tf.nn.softmax(tf.matmul(output, rnn_to_layer_arg_weight))
            chosen_layer_type = tf.placeholder(tf.int32, self.batch_size)
            chosen_layer_arg = tf.placeholder(tf.int32, self.batch_size)
            self.layer_probs.append((layer_type, layer_arg))
            self.layer_indicators.append((chosen_layer_type, chosen_layer_arg))

            # aggregate gradients
            baseline = self.reward_ema.average(self.last_average_reward)
            prob = (self.last_average_reward - baseline) * (
                tf.reduce_sum(
                    tf.one_hot(chosen_layer_type, depth=self.type_size) * tf.log(layer_type + 1e-12),
                    axis=1
                ) + tf.reduce_sum(
                    tf.one_hot(chosen_layer_arg, depth=self.arg_size) * tf.log(layer_arg + 1e-12),
                    axis=1
                )
            )
            losses.append(prob)

            rnn_to_layer_arg_gradient.append(tf.gradients(prob, rnn_to_layer_arg_weight)[0])
            rnn_to_layer_type_gradient.append(tf.gradients(prob, rnn_to_layer_type_weight)[0])
            for param, grad in zip(rnn_params, rnn_gradients):
                grad.append(tf.gradients(prob, param)[0])

        self.loss = tf.reduce_mean(losses)

        def sanitize_gradient(grads):
            avg = sum(grads) / len(grads)
            return tf.clip_by_norm(avg, 1.0)

        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        self.optimize = optimizer.apply_gradients([
            (sanitize_gradient(grad), param)
            for param, grad in zip(rnn_params, rnn_gradients)
        ] + [
            (sanitize_gradient(rnn_to_layer_type_gradient), rnn_to_layer_type_weight),
            (sanitize_gradient(rnn_to_layer_arg_gradient), rnn_to_layer_arg_weight),
        ])

    def generate_architecture(self, session):
        layers = session.run(self.layer_probs)
        networks = [[] for _ in range(self.batch_size)]
        for (ltype, larg) in layers:
            for i, (nnet, type_prob, arg_prob) in enumerate(zip(networks, ltype, larg)):
                # the network always has at least one layer
                if nnet and nnet[-1][0] == 0:
                    continue

                assert all(np.isfinite(type_prob))
                assert all(np.isfinite(arg_prob))

                layer_type = np.random.choice(len(type_prob), p=type_prob)
                layer_arg = np.random.choice(len(arg_prob), p=arg_prob)
                nnet.append((layer_type, layer_arg))

        return networks

    def learn_from_rewards(self, sess, networks, rewards):
        assert len(rewards) == self.batch_size

        # set the indicator variables, telling which action was chosen
        feed_dict = {ind: []
                     for layer_ind in self.layer_indicators
                     for ind in layer_ind}

        for nnet in networks:
            # pad network if shorter than expected
            # we set the indicators to -1, so that all one hot will be 0
            # thus not contributing to the gradient
            if len(nnet) < self.unroll_by:
                nnet = nnet + [(-1, -1)] * (self.unroll_by - len(nnet))

            assert len(nnet) == self.unroll_by
            for (itype, iarg), (ntype, narg) in zip(self.layer_indicators, nnet):
                feed_dict[itype].append(ntype)
                feed_dict[iarg].append(narg)

        feed_dict[self.architecture_reward] = rewards
        loss, _, _ = sess.run([self.loss, self.update_reward_ema, self.optimize],
                              feed_dict=feed_dict)
        return loss

    
class MovingAverages:
    def __init__(self):
        self.metrics = {}
        self.smoothing = {}
        self.snapshots = []
    
    def update(self, metric, value, smoothing=None):
        if smoothing is None:
            smoothing = self.smoothing.get(metric, 0.6)
        self.smoothing[metric] = smoothing
        
        # can pass None to update smoothing
        if value is not None:
            self.metrics[metric] = (
                smoothing * self.metrics.get(metric, value)
                + (1 - smoothing) * value
            )
        return self.metrics[metric]
    
    def update_all(self, **metrics):
        for metric, value in metrics.items():
            self.update(metric, value)
        return [self.metrics[m] for m in metrics]
    
    def snapshot(self, **meta):
        snap = dict(self.metrics)
        snap.update(meta)
        self.snapshots.append(snap)
        return snap

    

def get_train_test_data(df, features, target, samples_count, n_months=12):
    df = df.dropna()

    # get random test months
    test_ds = np.random.choice(df.ds.unique(), n_months, replace=False)
    test_mask = df.ds.isin(test_ds)
    
    train_df, test_df = df.loc[~test_mask], df.loc[test_mask]
    if samples_count > 0:
        # maintain proportion of train/test samples
        test_size = int(samples_count * len(test_df) / len(train_df))
        train_df = train_df.sample(samples_count)
        test_df = test_df.sample(test_size)
    
    train_x, train_y = train_df[features], train_df[target]
    test_x, test_y = test_df[features], test_df[target]

    mean_x, mean_y = train_x.mean(), train_y.mean()
    std_x, std_y = train_x.std(), train_y.std()

    train_x = (train_x - mean_x) /  std_x
    test_x = (test_x - mean_x) / std_x
    
    assert np.all(np.isfinite(train_x))
    
    train_y = (train_y - mean_y) / std_y
    test_y = (test_y - mean_y) / std_y

    return train_x, train_y, test_x, test_y, mean_y, std_y
    

def compute_denormalized_mse(std_y):
    def denormalized_mse(y_true, y_pred):
        # model is trained with normalized data, but we want
        # mse on not normalized data to compare with MOST
        mse = K.mean(K.square(y_true - y_pred), axis=-1)
        return mse * std_y**2
    return denormalized_mse


def build_model(input_shape, architecture, std_y=1):    
    regularizer = None
    layers = [Input(shape=(input_shape,))]

    for layer_type, layer_arg in architecture:
        if layer_type == 0 or layer_type == 1:
            num = 2**layer_arg
            layers.append(PReLU()(
                BatchNormalization()(
                    Dense(num, kernel_initializer=VarianceScaling(2, 'fan_in'),
                          kernel_regularizer=regularizer)(
                        layers[-1]
                    )
                )
            ))
        elif layer_type == 2:
            pkeep = (layer_arg + 1) / 10
            layers.append(Dropout(pkeep)(layers[-1]))
        elif layer_type == 3:
            regu = 6 ** -layer_arg
            regularizer = regularizers.l2(regu)
        else:
            raise ValueError('layer type from 0 to 3')

    layers.append(Dense(1)(layers[-1]))

    opt = Adam(lr=0.001)
    model = Model(inputs=layers[0], outputs=layers[-1])
    model.compile(loss='mse', optimizer=opt, metrics=[compute_denormalized_mse(std_y)])
    return model


def evaluate_architecture(step, arch_idx, architecture, max_epochs, samples_count):
    train_x, train_y, test_x, test_y, _, std_y = get_train_test_data(
        df, [c for c in df.columns if c != 'phi_m' and c != 'ds'], 'phi_m', samples_count, n_months=12
    )
    
    #K.clear_session()  # https://stackoverflow.com/q/35114376/521776
    model = build_model(train_x.shape[1], architecture, std_y=std_y)

    logdir = 'dev/logs/nas/step-%d-arch-%d' % (step, arch_idx)
    callbacks = [
        ReduceLROnPlateau(factor=0.1, verbose=0, min_lr=1e-6, patience=10),
        TensorBoard(logdir, write_graph=True, write_grads=True, histogram_freq=0),
        EarlyStopping(min_delta=0.001, patience=25),
    ]

    hist = model.fit(
        train_x, train_y,
        batch_size=1024,
        epochs=max_epochs,
        verbose=0,
        shuffle=True,
        callbacks=callbacks,
        validation_data=(test_x, test_y)
    )

    best = min(hist.history['val_denormalized_mse'])

    return best

In [4]:
controller = ControllerRNN(
    hidden_size=64,
    max_len=20,
    batch_size=1,
    type_size=3,
    arg_size=10,
    learning_rate=0.001,
    baseline_smoothing=0.99,
)

hist = []
controller_graph = tf.Graph()
with controller_graph.as_default():
    controller.build()
    controller_session = tf.Session(graph=controller_graph)
    controller_session.run(tf.global_variables_initializer())

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [ ]:
averages = MovingAverages()
averages.smoothing['time'] = 0
start_time = time.time()
for step_idx in range(10000):
    avg_mse = averages.metrics.get('inner_mse', 10)
    if avg_mse > 1:
        samples_count, max_epochs = 250000, 5
    elif avg_mse > 0.5:
        samples_count, max_epochs = 500000, 20
    elif avg_mse > 0.3:
        samples_count, max_epochs = -1, 50
    else:
        samples_count, max_epochs = -1, 500

    architectures = controller.generate_architecture(controller_session)

    # test architectures on a temporary graph
    with tf.Graph().as_default():
        with tf.Session().as_default():
            rewards = [
                evaluate_architecture(
                    step_idx, arch_idx, arch,
                    max_epochs, samples_count
                )
                for arch_idx, arch in enumerate(architectures)
            ]

    loss = controller.learn_from_rewards(
        controller_session, architectures, rewards
    )

    averages.update_all(
        contro_loss=loss**2,
        inner_mse=np.mean(rewards),
        inner_mse_std=np.std(rewards),
        length=np.mean(list(map(len, architectures))),
        fc_count=np.mean([sum(1 for lt, _ in arch if lt <= 1) for arch in architectures]),
        dropo_count=np.mean([sum(1 for lt, _ in arch if lt == 2) for arch in architectures]),
        regu_count=np.mean([sum(1 for lt, _ in arch if lt == 3) for arch in architectures]),
    )

    if step_idx % 1 == 0:
        snap = averages.snapshot(step=step_idx, time=time.time() - start_time)
        print('  '.join(
            '%s=%.3f' % metric for metric in snap.items()
        ))

contro_loss=83.979  inner_mse=2.065  inner_mse_std=0.000  length=10.000  fc_count=1.000  dropo_count=9.000  regu_count=0.000  step=0.000  time=9.453
contro_loss=80.164  inner_mse=2.158  inner_mse_std=0.000  length=14.000  fc_count=1.000  dropo_count=13.000  regu_count=0.000  step=1.000  time=32.737
contro_loss=48.230  inner_mse=1.463  inner_mse_std=0.000  length=8.800  fc_count=1.000  dropo_count=7.800  regu_count=0.000  step=2.000  time=39.806
contro_loss=28.977  inner_mse=1.050  inner_mse_std=0.000  length=5.680  fc_count=1.000  dropo_count=4.680  regu_count=0.000  step=3.000  time=46.790
contro_loss=3902.494  inner_mse=1.886  inner_mse_std=0.000  length=11.408  fc_count=0.600  dropo_count=10.008  regu_count=0.800  step=4.000  time=55.917
contro_loss=3854.596  inner_mse=2.416  inner_mse_std=0.000  length=14.845  fc_count=1.160  dropo_count=13.205  regu_count=0.480  step=5.000  time=82.531
contro_loss=2312.823  inner_mse=1.580  inner_mse_std=0.000  length=9.307  fc_count=1.096  dropo_